# Modelo 4

Vamos aplicar os modelos de Machine Learning para os dados relacionados ao videos do Youtube. É importante ressaltar que foram revistos 0 e 1 aplicados a coluna 'y' para que tenha menos carga aleatória. Além disso, foi preenchido manualmente todas as informações de acordo com o título.

In [1]:
#Importando as bibliotecas
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('raw_data_with_label_rev1.xlsx')

## Tratamento dos dados

Nesta etapa vamos retirar linhas repetidas a onde um mesmo video aparece para duas palavras-chaves (ex: Machine Learning e Kaggle).

Além disso, vamos verificar os valores ausentes.

In [3]:
#Deletando as linhas duplicadas
df.drop_duplicates(inplace = True)

In [4]:
#Verificando a quantidade de nulos
df.isnull().sum()

title          0
y              0
upload_date    2
view_count     2
query          2
dtype: int64

In [5]:
#Deletando as linhas que estão com valores ausentes
df.dropna(axis = 0, subset = ['upload_date', 'view_count', 'query'], inplace = True)

In [6]:
#Convertendo a variável date para datetime
df['upload_date'] = pd.to_datetime(df['upload_date'])

## Features

Esta etapa está associada a criação de features

In [7]:
#Armazenando os dados com y preenchido
df_preenchido = df.loc[ df['y'].notnull() ]

In [8]:
#Criando um dataframe vazio para features com o mesmo índices do df_preenchido
features = pd.DataFrame(index = df_preenchido.index)

#Criando um dataframe exclusivo para y -> Variável Alvo
y = df_preenchido['y'].copy()

**Criando uma feature de data desde da publicação**

Como os modelos de Machine Learning não conseguem ler datas, devemos criar alguma forma para transformar estas datas em dados numéricos. Portanto, iremos fazer a diferença em relação a data de hoje (31/05/2021)

In [9]:
pd.to_datetime('2021/05/31')-df_preenchido['upload_date']

0        5 days
1        5 days
2        5 days
3        5 days
4        5 days
         ...   
1505   915 days
1506   938 days
1507   942 days
1508   950 days
1509   951 days
Name: upload_date, Length: 1507, dtype: timedelta64[ns]

Como podemos observar vem com a palavras "days" e no formato timedelta64. Para contornarmos isto, utilizaremos um método do numpy chamado **timedelta64** em que passaremos como argumento a diferença de um dia, já que os dados estão associados a diferença diária. Isto permite que os valores sejam convertidos para float.

In [10]:
(pd.to_datetime('2021/05/31')-df_preenchido['upload_date'])/np.timedelta64(1, 'D')

0         5.0
1         5.0
2         5.0
3         5.0
4         5.0
        ...  
1505    915.0
1506    938.0
1507    942.0
1508    950.0
1509    951.0
Name: upload_date, Length: 1507, dtype: float64

In [11]:
#Criando a feature de diferença diária
features['tempo_desde_pub'] = (pd.to_datetime('2021/05/31')-df_preenchido['upload_date'])/np.timedelta64(1, 'D')

**Features - Views**
Criando um conjunto de features para view.

Iremos replicar o count_views e iremos calcular as views por dia, utilizando a feature tempo_desde_pub

In [12]:
#Armazenando as views
features['views'] = df_preenchido['view_count']

#Fazendo o cálculo de views por dia
features['views_por_dia'] = features['views'] / features['tempo_desde_pub'] # Serve para avaliar a capacidade do video em promover views
                                                                            # E desconsiderar o fator tempo

Como é uma séria temporal e os dados devem ser fatiados considerando a sequência temporal, a feature **tempo_desde_pub** vai trazer a mesma informação do que a variável **date**, ou seja, se fatiássemos em 01/01/2020, os valores antes desta data seriam menores e após seriam maiores, porque seguem a mesma sequência temporal.

Diante disto, vamos deletar esta coluna do dataset

In [13]:
#Deletando a variável tempo_desde_pub
features.drop(['tempo_desde_pub'], axis = 1, inplace = True)

In [14]:
#Imprimindo as 5 primeiras linhas
features.head()

,views,views_por_dia
0,7.0,1.4
1,27.0,5.4
2,1.0,0.2
3,20.0,4.0
4,2.0,0.4


## Preparação dos Dados

In [15]:
#Separando entre treino e teste
mask_train = df_preenchido['upload_date'] < "2020-04-03" #Utilizado +- a metade do período temporal
mask_val = df_preenchido['upload_date'] >= "2020-04-03"

X_train, X_val = features[mask_train], features[mask_val]
y_train, y_val = y[mask_train], y[mask_val]
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((515, 2), (992, 2), (515,), (992,))

Vamos utilizar o método **TfidfVectorizer** para extrair palavras do título. Este método é bom, porque ele atribui maior peso as palavras que aparecem menos.

Além disso, utilizamos este método para transformar informações textuais em números.

Utilizaremos o argumento min_df = 2 para que seja considerada como coluna apenas as palavras que aparecerem 2 vezes ou mais no dataset. 

In [48]:
#Importando o método
from sklearn.feature_extraction.text import TfidfVectorizer

#Criando uma variável de títulos com os dados de treino e validação
title_train = df_preenchido.loc[mask_train, 'title']
title_val = df_preenchido.loc[mask_val, 'title']

#Instanciando o objeto TfidfVectorizer
title_vec = TfidfVectorizer(min_df = 2, ngram_range = (1,1))

#Treinando e transformando os dados de treino referente ao título
title_bow_train = title_vec.fit_transform(title_train)

#Transformando os dados de validação com os parâmetros de treino
title_bow_val = title_vec.transform(title_val)

#o termo bow significa bag of words -> devido a matriz de palavras geradas

Os resultados gerados após aplicação do método é uma matriz esparsa que possui a seguinte definição:
* Quando possui matrizes em que existem muitos zeros, a matriz esparsa apenas armazena (na memória) os valores diferentes de zero. Portanto, quando ela for consultar um elemento e ele não estiver armazenado, retorna zero.

O próximo passo é unirmos os dados de título com o X_train e X_val. Para isto, utilizaremos um método do Scipy chamado hstack e funciona da seguinte maneira:

*Mostraremos o funcionamento também da vstack*

hstack - [1 2]     [3 4]   -> [1 2 3 4] - 1x4 -> Junta os elementos em colunas 

vstack - [1 2]     [3 4]   -> [1 2]
                              [3 4] - 2x2 -> Junta os elementos em linha
                              
Não utilizar estes métodos do numpy, apenas do scipy sparse, porque o numpy não está otimizado para trabalhar com matrizes esparsas, com isso, o tempo de aplicação do método seria maior do que scipy. 

In [50]:
#Importando o método hstack
from scipy.sparse import hstack

#Aplicando o método hstack para juntar o X com as matrizes de bag of word
X_train_wtitle = hstack([X_train, title_bow_train])
X_val_wtitle = hstack([X_val, title_bow_val])

In [51]:
#Imprimindo as dimensões
X_train_wtitle.shape, X_val_wtitle.shape

((515, 433), (992, 433))

### Random Forest

In [19]:
#Importando as bibliotecas
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

In [20]:
#Instanciando a RandomForest
#Como os dados estão desbalanceados, vamos utilizar o argumento class_weight para dar mais peso a classe desbalanceada
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf = 1, class_weight="balanced", n_jobs=6)
mdl.fit(X_train_wtitle, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [21]:
#Pegando as probabilidade
p = mdl.predict_proba(X_val_wtitle)[:,1]

In [22]:
#Calculando a metrica average_precision_score
average_precision_score(y_val, p)

0.5758634597636195

In [23]:
#Calculando a ROC_AUC
roc_auc_score(y_val, p)

0.7489579818847162

Vamos tunar os o parâmetros **TfidfVectorizer** e **Random Forest** para verificar se as métricas melhoram ou não. Neste caso, vamos apenas modificar os valores dos códigos acima e registrar os valores obtidos nas métricas.

Para **TfidfVectorizer** simularemos os seguintes parâmetros:
* min_df -> A frequência mínima que a palavra deve ter para contabilizar como coluna
* ngram_range -> Representado por uma tupla em que a primeira posição está associado ao mínimo de gramas e a última ao máximo de gramas. Segue um exemplo para ilustras: intro to machine learning -> intro, to, machine, learning -> ngram_range=(1,1) 
                       intro to machine learning -> intro, to, machine, learning, intro to, to machine, machine learning -> ngram_range=(1,2)
                       intro to machine learning -> intro to, to machine, machine learning -> ngram_range=(2,2)
                       
É importante destacar que o ngrama é uma palavra.

Para o **Random Forest** usaremos:
* n_estimators: Número de árvores -> 100 ou 1000
* min_samples_leaf: Mostra quantos dados devem ter nos nós finais das árvores.

Resultado:
* Padrão (min_df = 2, n_estimators = 1000) -> ROC_AUC = 0.7551062780618204 e AP = 0.581860377926327
* Parâmetros (min_df = 2, n_estimators = 1000, min_sample_lif = 2) -> ROC_AUC = 0.7678285017258831 e AP = 0.575313410802137
* Parâmetros (min_df = 2, n_estimators = 100, min_sample_lif = 2) -> ROC_AUC = 0.7614621992681218 e AP = 0.5736100883158195
* Parâmetros (min_df = 2, n_estimators = 100, min_sample_lif = 1) -> ROC_AUC = 0.7370870208403624 e AP = 0.5614679067140127
* Parâmetro (min_df = 1, n_estimators = 1000) -> ROC_AUC = 0.7530404090213075 e AP = 0.5588239160420265
* Parâmetro (min_df = 3, n_estimators = 1000) -> ROC_AUC = 0.7489579818847162 e AP = 0.5758634597636195
* Parâmetro (min_df = 2, n_estimators = 1000, ngram (1,2)) -> ROC_AUC = 0.7385455866704731 e AP = 0.5768589581963872
* Parâmetro (min_df = 2, n_estimators = 1000, ngram (1,3)) -> ROC_AUC = 0.7212478264254756 e AP = 0.5635050362865264
* Parâmetro (min_df = 2, n_estimators = 1000, ngram (1,4)) -> ROC_AUC = 0.7201370325192702 e AP = 0.5612889724473165

Apesar do ROC_AUC melhorar um pouco nas mudanças de parâmetros, vamos continuar com os parâmetros iniciais, porque conseguimos manter uma boa average_precision_recall

* Padrão (min_df = 2, n_estimators = 1000) -> ROC_AUC = 0.7551062780618204 e AP = 0.581860377926327

### LigthGBM

Parâmetros importantes:
* Learning_rate -> Basicamente significa a contribuição de cada árvore para combinação de árvore que o modelo vai criar. Neste parâmetro estamos mais preocupados em buscar valores menores.
* num_leaves -> Número máximo de folhas, ou seja, de nós finais. Quanto maior for este número, mais complexo será o modelo e é uma forma de limitar o modelo de não memorizar os dados e aprender os padrões que achemos necessário. Este parâmetro não é a mesma coisa do que profundidade, porque nem sempre a árvore vai ser balanceada, ou seja, o lado direito e esquerdo terminarão no mesmo nível de profundidade.
* min_child_sample -> Número mínimo de amostras por nó. Este é um outro parâmetro que ajuda a evitar que o modelo memorize os dados
* subsample -> Cada vez que for criar uma árvore não vai ser sob todos os dados, este parâmetro serve para definir as amostras que serão criadas para cada árvore. Ajuda a dar diversidade a árvore e evitar a memorização dos dados. Devemos sempre preencher a subsample_freq que significa a quantas árvores devemos amostrar os dados.
* colsample_bytree -> É uma forma de não deixar todas as variáveis disponíveis para todas as árvores. É similar ao subsample em que é disponibilizado uma amostra de variáveis para cada árvore. Ajuda também a dar diversidade a árvore e evitar a memorização dos dados.

Formas de tunar os hiperparâmetros, utilizando a biblioteca scikit-optimize:
* dummy_minimize -> É preciso criar uma função (def) para treinar o modelo a partir dos parâmetros escolhidos. Devemos passar para o dummy_minimize esta função, quantidade de iterações e uma lista dos parâmetros em que cada parâmetros deve ser informado através de uma tupla com o valor mínimo e máximo para variação. Podemos também informar a distribuição para gerar pesos diferentes, por default, é uniforme. A ideia desta função é rodar de forma aleatória várias combinações de parâmetros até que minimize uma função 'custo' que deve ser definida como retorno na função criada inicialmente. Vale destacar que os scores utilizados são na maioria quanto maior melhor, por isso, para estes casos devemos utilizar, por exemplo, - roc_auc_score.

* bayesiana_optimizacion (gp_minimize - utiliza processos gaussianos e forest_minimize - utiliza random forest. A escolha depende da quantidade de parâmetros para tunar ) -> Ela também roda de forma aleatória, porém, é mais guiada. Com isso, avalia o impacto do score quando mudamos os parâmetros dentro do modelo escolhido. Então, ela avalia a sensibilidade das mudanças destes parâmetros no modelo escolhido, priorizando um caminho de maior impacto para maximizar o score. Portanto, alguns parâmetros serão considerados mais importantes do que outros. O problema da utilização do GridSearch que se perde muito tempo avaliando parâmetros de pouco impacto sem uma busca mais inteligente, já que considera que todos parâmetros possuem mesmo impacto. Os argumentos de entrada são as mesmas da dummy_minimize, porém, deve acrescentar n_random_starts (geralmente é 10), basicamente serve para pegar 10 pontos aleatórios para avaliar a combinação de parâmetros melhor e ir guiando os próximos passos, tentando avaliar os parâmetros mais promissores (avaliar uma região de pontos e qual é o ponto ótimo. Diante disso, passa para o próximos 10 pontos e avaliar o próximo ponto ótimo). 

* Comentário do Mario Filho - Ele é a favor de testar tudo, porém, não devemos aplicar o GridSearch e o modelo Naive Bayes (Modelo bem fraquinho). Existe papers e artigos mostrando que a Random Search chega em uma resultado muito melhor em menos tempo do que a GridSearch. Além disso, alguns artigos mostram que bayesina optimization possui resultados iguais ou melhores que a Random Search. Lembrando que a Bayesiana Optimization funciona como fosse uma busca aleatória, porém, guiada pela sensibilidade do modelo em relação as range de parâmetros.

**IMPORTANTE** 
* Utilizar a forest_minimize quando tivermos mais de 5 parâmetros para tunar
* Menos do que este valor, podemos utilizar a gp_minimize



In [24]:
#Instalando scikit-optimize
!pip install scikit-optimize

In [25]:
#Instalando lighgbm
!pip install lightgbm

**Aplicação do LighGBM sem tunar os hiperparâmetros**

Primeiro vamos avaliar a performance do modelo sem a tunar os hiperparâmetros.

In [26]:
#Importando o modelo
from lightgbm import LGBMClassifier

#Instanciando o modelo
mdl_lgm = LGBMClassifier(random_state = 42, class_weight="balanced", n_jobs=6)

#Treinando o modelo
mdl_lgm.fit(X_train_wtitle, y_train)

#Pegando as probabilidade
p = mdl_lgm.predict_proba(X_val_wtitle)[:,1]

#Calculando a metrica average_precision_score
print("AP = {}".format(average_precision_score(y_val, p)))

#Calculando a ROC_AUC
print("ROC_AUC = {}".format(roc_auc_score(y_val, p)))

AP = 0.4851778075837254
ROC_AUC = 0.7070903947470868


C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


**Tunando os hiperparâmetros**

Iremos tunar tanto os hiperparâmetros do LighGBM quanto TfidfVectorizer,como fizemos na Random Forest.

O próximo passo é criarmos uma função que receba os parâmetros e busque minimizar o erro. Escolheremos como métrica principal a **average_precision**.

É importante destacar que os parâmetros são fornecidos a função aleatoriamente, seguindo uma distribuição uniforme. Porém, para o caso específico da **learning_rate**, utilizaremos uma distribuição log-uniforme para gerar um peso maior para os valores menores. Como vimos anteriormente, quanto menor for este valor melhor será.

Os parâmetros devem ser dados em forma de tupla, contendo o valor mínimo e máximo para variação, podendo também informar uma distribuição diferente da uniforme. Vale destacar que se dermos os parâmetros sem ponto, a variação será feita de forma inteira e com ponto de forma decimal. Exemeplo:
(100, 1000) -> Irá variar de forma inteira -> 100, 897, 465 e etc
(0.1, 1.) -> Irá variar de forma decimal -> 0.2, 0.78 e etc

In [27]:
#Criando a função
def tune_lgbm(params):
    
    #Imprimindo os parâmetros
    print(params)
    
    #Atribuindo os paramêtros LGBM
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    #Atribuindo os parâmetros para bag of words
    min_df = params[6]
    ngram_range = (1, params[7]) #Considerando como valor padrão mínimo a unigrama
    
    
    #----------------------------------------------Criando a Bag of Words com os parâmetros tunados-------------------------------
    
    #Instanciando o objeto TfidfVectorizer
    title_vec = TfidfVectorizer(min_df = min_df, ngram_range = ngram_range)

    #Treinando e transformando os dados de treino referente ao título
    title_bow_train = title_vec.fit_transform(title_train)

    #Transformando os dados de validação com os parâmetros de treino
    title_bow_val = title_vec.transform(title_val)
    
    #----------------------------------------------Juntando o X com as bag of words -------------------------------------------
    
    #Aplicando o método hstack para juntar o X com as matrizes de bag of word
    X_train_wtitle = hstack([X_train, title_bow_train])
    X_val_wtitle = hstack([X_val, title_bow_val])
    
    #---------------------------------------------Tunando os parâmetros e aplicando-------------------------------------------
    
    #Instanciando o modelo
    mdl_lgm = LGBMClassifier(random_state = 42, 
                             class_weight="balanced", 
                             n_jobs=6, 
                             learning_rate = lr,
                             num_leaves = 2 ** max_depth, #Para dar um sentido hipotético do tamanho da árvore, olhar comentário do Mario
                             max_depth = max_depth,
                             min_child_samples = min_child_samples,
                             subsample = subsample,
                             colsample_bytree = colsample_bytree,
                             n_estimators = n_estimators,
                             bagging_freq = 1)

    #Treinando o modelo
    mdl_lgm.fit(X_train_wtitle, y_train)

    #Pegando as probabilidade
    p = mdl_lgm.predict_proba(X_val_wtitle)[:,1]

    #Calculando a ROC_AUC
    print("ROC_AUC = {}".format(roc_auc_score(y_val, p)))
    
    #Retorno da função custo -> Como queremos minimizar, devemos atribuir o sinal de "-" a frente do score para funcionar
    return -average_precision_score(y_val, p)
    
    

In [28]:
#Importando o baysian optimize
from skopt import forest_minimize

#Setando os parâmetros em tuplas
space = [(1e-3, 1e-1, 'log-uniform'), # lr - Log-Uniform gera um peso maior para os valores menores, conforme dito anteriormente
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (1,5), # min_df
          (1,5)] # ngram_range

#Aplicando o tunning
res = forest_minimize(tune_lgbm, space, random_state=42, n_random_starts=20, n_calls=50, verbose=1)
# n_random_starts -> Pelo o que entendi, quantos eventos precisa reunir de forma aleatória para avaliar o ponto ótimo
# n_calls -> quantidade de vezes que vai ser aplicado

Iteration No: 1 started. Evaluating function at random point.
[0.03918194347141743, 8, 7, 0.4735411152109116, 0.14497617002710275, 558, 5, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


ROC_AUC = 0.7192208870779373
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2393
Function value obtained: -0.5215
Current minimum: -0.5215
Iteration No: 2 started. Evaluating function at random point.
[0.0019307837536547132, 3, 2, 0.7358988336534836, 0.9416250735649627, 485, 4, 5]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
ROC_AUC = 0.7528691183722196
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.1788
Function value obtained: -0.6059
Current minimum: -0.6059
Iteration No: 3 started. Evaluating function at random point.
[0.017177621112338382, 10, 12, 0.07190930378934497, 0.5485359272454697, 610, 2, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


ROC_AUC = 0.6220238249721004
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.2224
Function value obtained: -0.4460
Current minimum: -0.6059
Iteration No: 4 started. Evaluating function at random point.
[0.08861577452533079, 3, 12, 0.538522716492931, 0.6127938404189405, 230, 5, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
ROC_AUC = 0.7184319119669876
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0816
Function value obtained: -0.5153
Current minimum: -0.6059
Iteration No: 5 started. Evaluating function at random point.
[0.022941144328643994, 9, 7, 0.06260171310187321, 0.9450916679006103, 113, 2, 2]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
ROC_AUC = 0.6471153097505905
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0671
Function value obtained: -0.4338
Current minimum: -0.6059
Iteration No: 

...

ROC_AUC = 0.7665178687290755
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 0.4288
Function value obtained: -0.5924
Current minimum: -0.6402
Iteration No: 49 started. Searching for the next optimal point.
[0.010127818745010784, 9, 1, 0.4765189617330622, 0.9031844020096405, 474, 5, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


ROC_AUC = 0.7658923983286186
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 0.6836
Function value obtained: -0.5927
Current minimum: -0.6402
Iteration No: 50 started. Searching for the next optimal point.
[0.0022855320267921395, 6, 1, 0.4445021474318012, 0.9804824738416845, 150, 2, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
ROC_AUC = 0.7620150009083595


C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.3472
Function value obtained: -0.5971
Current minimum: -0.6402


In [29]:
#Melhor hiperparâmetros
res.x

[0.005607394468548006, 7, 1, 0.6655993304551099, 0.9586294865021325, 523, 2, 1]

In [30]:
#Melhor score da average precision
res.fun

-0.6402330807008632

In [31]:
#Vamos replicaro os parâmetros tunados
ap = tune_lgbm(res.x)

#Imprimindo o valor
print("Average_precision = {}".format(-ap))

[0.005607394468548006, 7, 1, 0.6655993304551099, 0.9586294865021325, 523, 2, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
ROC_AUC = 0.7966338792141392
Average_precision = 0.6402330807008632


C:\Users\fabio\anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


Até agora conseguimos os seguintes valores:

* LGBM tunado -> ROC_AUC = 0.7966338792141392 e Average_precision = 0.6402330807008632
* Random Forest tunado -> ROC_AUC = 0.7551062780618204 e AP = 0.581860377926327

Não aplicamos este formato de tunning na Random Forest, porque, segundo o Mario Filho, este modelo são menos sensíveis a mudanças dos parâmetros.

#### Regressão Logística

Como a função de otimização deste modelo é baseada no **Gradiente Descendente**, será necessário escalonar as variáveis numéricas com o intuito de manter todas as variáveis na mesma escalara. Para isto, utilizaremos dois métodos em busca do mais eficiente:
* StandarScaler -> É um dos mais usuais, utilizamos para buscar uma escala com média igual a zero e desvio padrão igual. 
* MaxAbsScaler -> Divide todos os valores pelo maior valor da coluna -> ex [100,500,1000,50] = [100 / 10000, 500 / 1000, 1000 / 1000, 50 / 1000]

Como as matrizes são dadas em matriz esparsas, será necessário aplicar também um método chamado de **csr_matrix** para termos acesso as colunas para o escalonamento das variáveis numéricas.

O próximo passo é aplicarmos a Regressão Logística sem tunning.

In [32]:
#Importando as bibliotecas
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression

In [60]:
#Intânciando os scalers
scaler_standard = StandardScaler()
scaler_max_abs = MaxAbsScaler()

#Fazendo um for para cada scaler
for scaler in [scaler_standard, scaler_max_abs]:
        
    #Criando um cópia do X_train e X_val com csr_matrix para manipulação de variáveis durante o scaler
    X_train_wtitle2 = csr_matrix(X_train_wtitle.copy())
    X_val_wtitle2 = csr_matrix(X_val_wtitle.copy())
    
    if scaler == scaler_standard:
    
        #Escalonando as variáveis numéricas
        X_train_wtitle2[: , :2] = scaler.fit_transform(X_train_wtitle2[:, :2].todense()) #Matriz densa é o oposto de matriz esparsa
        X_val_wtitle2[: , :2] = scaler.transform(X_val_wtitle2[:, :2].todense())
        
    else :
        
        #Escalonando todas as variáveis - Como pega o valor máximo por coluna não gera problema quando for escalonar as variáveis categóricas
        X_train_wtitle2 = scaler.fit_transform(X_train_wtitle2.todense()) #Matriz densa é o oposto de matriz esparsa
        X_val_wtitle2 = scaler.transform(X_val_wtitle2.todense())
    
    #Instanciando o modelo
    mdl = LogisticRegression(n_jobs = 6, random_state = 42, class_weight="balanced")
    
    #Treinando o modelo
    mdl.fit(X_train_wtitle2, y_train)
    
    #Prevendo o modelo
    p = mdl.predict_proba(X_val_wtitle2)[:, 1]
    
    #Calculando os scores
    print("{} - AP = {}".format(str(scaler), average_precision_score(y_val, p) ))
    print("{} - ROC_AUC = {}".format(str(scaler), roc_auc_score(y_val, p) ))


C:\Users\fabio\anaconda3\lib\site-packages\scipy\sparse\_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


StandardScaler() - AP = 0.5747056828293885
StandardScaler() - ROC_AUC = 0.7719913835612883
MaxAbsScaler() - AP = 0.586897637438734
MaxAbsScaler() - ROC_AUC = 0.7700033739067244


**Sem tunning**
* StandardScaler() - AP = 0.5747056828293885 e ROC_AUC = 0.7705587708598272
* MaxAbsScaler() - AP = 0.586897637438734 e ROC_AUC = 0.7700033739067244

O próximo passo é tentarmos tunar os hiperparâmetros 

In [70]:
#Criando a função
def tune_rgl(params):
    
    #Imprimindo os parâmetros
    print(params)
    
    #Criando um lista de parâmetros para solver, scaler e penalty
    solver_list = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    penalty_list = ['l1', 'l2', 'none']
    scaler_list = [StandardScaler(), MaxAbsScaler()]
    
    #Atribuindo os paramêtros LGBM
    c = params[0]
    solver = solver_list[ params[1] ]
    penalty = penalty_list[ params[2] ]
    
    #Atribuindo os parâmetros para bag of words - Forçar para ficar na mesma referência dos últimos modelos e ser menos complexo o Deploy, assim evita ter que criar mais de um Vectorizer se a Regressão Logística for escolhida para o ensemble
    min_df = 2
    ngram_range = (1, 1) #Considerando como valor padrão mínimo a unigrama
    
    #Atribuindo os parâmetros scaler
    scaler = scaler_list[ params[3] ]
    
    
    #----------------------------------------------Criando a Bag of Words com os parâmetros tunados-------------------------------
    
    #Instanciando o objeto TfidfVectorizer
    title_vec = TfidfVectorizer(min_df = min_df, ngram_range = ngram_range)

    #Treinando e transformando os dados de treino referente ao título
    title_bow_train = title_vec.fit_transform(title_train)

    #Transformando os dados de validação com os parâmetros de treino
    title_bow_val = title_vec.transform(title_val)
    
    
    #----------------------------------------------Juntando o X com as bag of words -------------------------------------------
    
    #Aplicando o método hstack para juntar o X com as matrizes de bag of word
    X_train_wtitle = hstack([X_train, title_bow_train])
    X_val_wtitle = hstack([X_val, title_bow_val])
    
    #----------------------------------------------Escalonando as variáveis numéricas -----------------------------------------
    
    #Criando um cópia do X_train e X_val com csr_matrix para manipulação de variáveis durante o scaler
    X_train_wtitle2 = csr_matrix(X_train_wtitle.copy())
    X_val_wtitle2 = csr_matrix(X_val_wtitle.copy())
    
    if scaler == StandardScaler():
    
        #Escalonando as variáveis numéricas
        X_train_wtitle2[: , :2] = scaler.fit_transform(X_train_wtitle2[:, :2].todense()) #Matriz densa é o oposto de matriz esparsa
        X_val_wtitle2[: , :2] = scaler.transform(X_val_wtitle2[:, :2].todense())
        
    else :
        
        #Escalonando todas as variáveis - Como pega o valor máximo por coluna não gera problema quando for escalonar as variáveis categóricas
        X_train_wtitle2 = scaler.fit_transform(X_train_wtitle2.todense()) #Matriz densa é o oposto de matriz esparsa
        X_val_wtitle2 = scaler.transform(X_val_wtitle2.todense())
    
    #---------------------------------------------Tunando os parâmetros e aplicando-------------------------------------------
    
    #Alguns conjunto de parâmetros não funcionam juntos, por isto, vamos criar uma exception
    try:
    
        #Instanciando o modelo
        mdl_rlg = LogisticRegression(n_jobs = 6, 
                                     random_state = 42, 
                                     class_weight="balanced",
                                     C = c,
                                     solver = solver,
                                     penalty = penalty)

        #Treinando o modelo
        mdl_rlg.fit(X_train_wtitle2, y_train)

        #Pegando as probabilidade
        p = mdl_rlg.predict_proba(X_val_wtitle2)[:,1]

        #Calculando a ROC_AUC
        print("ROC_AUC = {}".format(roc_auc_score(y_val, p)))

        #Retorno da função custo -> Como queremos minimizar, devemos atribuir o sinal de "-" a frente do score para funcionar
        return -average_precision_score(y_val, p)
    
    except:
        
        #Cria um valor muito ruim para não ser escolhido com ponto ótimo
        return 0
    
    

In [72]:
#Importando o baysian optimize
from skopt import gp_minimize

#Setando os parâmetros em tuplas
space = [(1e-2, 100., 'log-uniform'), # C
          (0, 4), # solver
          (0, 2), # penalty
          (0,1)] #scaler 

#Aplicando o tunning
res = gp_minimize(tune_rgl, space, random_state=42, n_random_starts=20, n_calls=50, verbose=1)
# n_random_starts -> Pelo o que entendi, quantos eventos precisa reunir de forma aleatória para avaliar o ponto ótimo
# n_calls -> quantidade de vezes que vai ser aplicado

Iteration No: 1 started. Evaluating function at random point.
[15.35224694197351, 1, 2, 1]


C:\Users\fabio\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


ROC_AUC = 0.6945394617321119
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.6290
Function value obtained: -0.4107
Current minimum: -0.4107
Iteration No: 2 started. Evaluating function at random point.
[0.6071989493441302, 0, 1, 0]
ROC_AUC = 0.7294230619501181
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.1720
Function value obtained: -0.4939
Current minimum: -0.4939
Iteration No: 3 started. Evaluating function at random point.
[0.03727925903376984, 3, 0, 1]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0239
Function value obtained: 0.0000
Current minimum: -0.4939
Iteration No: 4 started. Evaluating function at random point.
[56.78201970293135, 0, 2, 1]


C:\Users\fabio\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1320: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


ROC_AUC = 0.581067192650074
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.2943
Function value obtained: -0.3089
Current minimum: -0.4939
Iteration No: 5 started. Evaluating function at random point.
[2.796485951606247, 0, 0, 1]
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0259
Function value obtained: 0.0000
Current minimum: -0.4939
Iteration No: 6 started. Evaluating function at random point.
[0.3975977214318099, 0, 2, 0]


...

ROC_AUC = 0.581067192650074
Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 2.2332
Function value obtained: -0.3089
Current minimum: -0.5818
Iteration No: 42 started. Searching for the next optimal point.
[0.9638851701068524, 4, 1, 1]
ROC_AUC = 0.7701279489242429
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.5336
Function value obtained: -0.5877
Current minimum: -0.5877
Iteration No: 43 started. Searching for the next optimal point.
[0.5888341100374204, 2, 1, 1]
ROC_AUC = 0.7701383301757027


C:\Users\fabio\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1353: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  warnings.warn("'n_jobs' > 1 does not have any effect when"


Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.3949
Function value obtained: -0.5857
Current minimum: -0.5877
Iteration No: 44 started. Searching for the next optimal point.
[1.1775685394292426, 1, 1, 1]
ROC_AUC = 0.7695673613454102
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 1.0339
Function value obtained: -0.5859
Current minimum: -0.5877
Iteration No: 45 started. Searching for the next optimal point.
[0.3920267184345894, 0, 1, 1]
ROC_AUC = 0.7701227582985128
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.6258
Function value obtained: -0.5822
Current minimum: -0.5877
Iteration No: 46 started. Searching for the next optimal point.
[100.0, 4, 0, 1]


C:\Users\fabio\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


ROC_AUC = 0.756741325166749
Iteration No: 46 ended. Search finished for the next optimal point.
Time taken: 0.7711
Function value obtained: -0.5397
Current minimum: -0.5877
Iteration No: 47 started. Searching for the next optimal point.
[0.34305161564276565, 4, 1, 1]
ROC_AUC = 0.7701487114271626
Iteration No: 47 ended. Search finished for the next optimal point.
Time taken: 0.6662
Function value obtained: -0.5812
Current minimum: -0.5877
Iteration No: 48 started. Searching for the next optimal point.
[1.103880368995756, 3, 1, 1]
ROC_AUC = 0.769956658275155
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 0.4498
Function value obtained: -0.5867
Current minimum: -0.5877
Iteration No: 49 started. Searching for the next optimal point.
[100.0, 0, 0, 1]
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 0.5949
Function value obtained: 0.0000
Current minimum: -0.5877
Iteration No: 50 started. Searching for the next optimal point.
[7.

In [73]:
#Imprimindo os melhores parâmetros
res.x

[0.9638851701068524, 4, 1, 1]

In [75]:
#Vamos replicar os parâmetros tunados
ap = tune_rgl(res.x)

#Imprimindo o valor
print("Average_precision = {}".format(-ap))

[0.9638851701068524, 4, 1, 1]
ROC_AUC = 0.7701279489242429
Average_precision = 0.5877359562456916


O tunning melhorou muito pouco em relação ao sem tunning.

* Regressão Logística -> AP = 0.5877359562456916 e ROC_AUC = 0.7701279489242429
* LGBM tunado -> ROC_AUC = 0.7966338792141392 e Average_precision = 0.6402330807008632
* Random Forest tunado -> ROC_AUC = 0.7551062780618204 e AP = 0.581860377926327